# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from scipy.stats import linregress

# Import API key
from config import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [13]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

636

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [14]:
# Set variables for base url and units
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

# Setup URL query using the base url, api key, and units
query_url = f"{url}appid={api_key}&units={units}&q="

# Create lists to hold relevant data
cityname_list = []
cloudiness_list = []
country_list = []
date_list = []
humiditylist_list = []
latitude_list = []
longitude_list = []
temperature_list = []
windspeed_list = []
indexcounter = 0

# Print header for data retrieval
print("Retrieving data...")
print("------------------")

# Loop through list of cities
for eachcity in cities:
    
    # Perform a get request for each city
    jsonresponse = requests.get(query_url + eachcity).json()
    
    print(json.dumps(jsonresponse, indent=4, sort_keys=True))

Retrieving data...
------------------
{
    "base": "stations",
    "clouds": {
        "all": 45
    },
    "cod": 200,
    "coord": {
        "lat": 22.2855,
        "lon": 114.1577
    },
    "dt": 1644545428,
    "id": 1819729,
    "main": {
        "feels_like": 68.2,
        "humidity": 79,
        "pressure": 1022,
        "temp": 67.98,
        "temp_max": 69.96,
        "temp_min": 67.24
    },
    "name": "Hong Kong",
    "sys": {
        "country": "HK",
        "id": 2035800,
        "sunrise": 1644533860,
        "sunset": 1644574662,
        "type": 2
    },
    "timezone": 28800,
    "visibility": 10000,
    "weather": [
        {
            "description": "scattered clouds",
            "icon": "03d",
            "id": 802,
            "main": "Clouds"
        }
    ],
    "wind": {
        "deg": 135,
        "gust": 17,
        "speed": 11.01
    }
}
{
    "base": "stations",
    "clouds": {
        "all": 0
    },
    "cod": 200,
    "coord": {
        "lat": 28.658

{
    "base": "stations",
    "clouds": {
        "all": 2
    },
    "cod": 200,
    "coord": {
        "lat": 22.6783,
        "lon": -3.9836
    },
    "dt": 1644545681,
    "id": 2450173,
    "main": {
        "feels_like": 55.78,
        "grnd_level": 1004,
        "humidity": 31,
        "pressure": 1020,
        "sea_level": 1020,
        "temp": 58.75,
        "temp_max": 58.75,
        "temp_min": 58.75
    },
    "name": "Taoudenni",
    "sys": {
        "country": "ML",
        "sunrise": 1644562230,
        "sunset": 1644603000
    },
    "timezone": 0,
    "visibility": 10000,
    "weather": [
        {
            "description": "clear sky",
            "icon": "01n",
            "id": 800,
            "main": "Clear"
        }
    ],
    "wind": {
        "deg": 30,
        "gust": 18.72,
        "speed": 10.49
    }
}
{
    "cod": "404",
    "message": "city not found"
}
{
    "base": "stations",
    "clouds": {
        "all": 100
    },
    "cod": 200,
    "coord": {
 

{
    "base": "stations",
    "clouds": {
        "all": 27
    },
    "cod": 200,
    "coord": {
        "lat": 5.8808,
        "lon": -10.0467
    },
    "dt": 1644545684,
    "id": 2278158,
    "main": {
        "feels_like": 78.58,
        "grnd_level": 1007,
        "humidity": 90,
        "pressure": 1008,
        "sea_level": 1008,
        "temp": 76.96,
        "temp_max": 76.96,
        "temp_min": 76.96
    },
    "name": "Buchanan",
    "sys": {
        "country": "LR",
        "id": 2389,
        "sunrise": 1644562618,
        "sunset": 1644605523,
        "type": 1
    },
    "timezone": 0,
    "visibility": 10000,
    "weather": [
        {
            "description": "scattered clouds",
            "icon": "03n",
            "id": 802,
            "main": "Clouds"
        }
    ],
    "wind": {
        "deg": 168,
        "gust": 5.77,
        "speed": 4.16
    }
}
{
    "cod": "404",
    "message": "city not found"
}
{
    "base": "stations",
    "clouds": {
        "all

{
    "cod": "404",
    "message": "city not found"
}
{
    "base": "stations",
    "clouds": {
        "all": 0
    },
    "cod": 200,
    "coord": {
        "lat": -33.6971,
        "lon": -53.4616
    },
    "dt": 1644545397,
    "id": 3443061,
    "main": {
        "feels_like": 69.91,
        "grnd_level": 1011,
        "humidity": 79,
        "pressure": 1013,
        "sea_level": 1013,
        "temp": 69.55,
        "temp_max": 69.55,
        "temp_min": 69.55
    },
    "name": "Chui",
    "sys": {
        "country": "UY",
        "sunrise": 1644483886,
        "sunset": 1644532292
    },
    "timezone": -10800,
    "visibility": 10000,
    "weather": [
        {
            "description": "clear sky",
            "icon": "01n",
            "id": 800,
            "main": "Clear"
        }
    ],
    "wind": {
        "deg": 54,
        "gust": 21.65,
        "speed": 9.78
    }
}
{
    "base": "stations",
    "clouds": {
        "all": 0
    },
    "cod": 200,
    "coord": {
  

{
    "base": "stations",
    "clouds": {
        "all": 100
    },
    "cod": 200,
    "coord": {
        "lat": 53.9667,
        "lon": 121.9667
    },
    "dt": 1644545690,
    "id": 2012938,
    "main": {
        "feels_like": -0.54,
        "grnd_level": 965,
        "humidity": 68,
        "pressure": 1036,
        "sea_level": 1036,
        "temp": -0.54,
        "temp_max": -0.54,
        "temp_min": -0.54
    },
    "name": "Yerofey Pavlovich",
    "sys": {
        "country": "RU",
        "sunrise": 1644535282,
        "sunset": 1644569491
    },
    "timezone": 32400,
    "visibility": 10000,
    "weather": [
        {
            "description": "overcast clouds",
            "icon": "04d",
            "id": 804,
            "main": "Clouds"
        }
    ],
    "wind": {
        "deg": 343,
        "gust": 4.27,
        "speed": 2.46
    }
}
{
    "base": "stations",
    "clouds": {
        "all": 40
    },
    "cod": 200,
    "coord": {
        "lat": -54.8,
        "lon":

...

{
    "base": "stations",
    "clouds": {
        "all": 0
    },
    "cod": 200,
    "coord": {
        "lat": 46.844,
        "lon": 74.9804
    },
    "dt": 1644545880,
    "id": 1525798,
    "main": {
        "feels_like": -15.99,
        "humidity": 84,
        "pressure": 1028,
        "temp": -3.39,
        "temp_max": -3.39,
        "temp_min": -3.39
    },
    "name": "Balkhash",
    "sys": {
        "country": "KZ",
        "id": 8817,
        "sunrise": 1644545484,
        "sunset": 1644581843,
        "type": 1
    },
    "timezone": 21600,
    "visibility": 10000,
    "weather": [
        {
            "description": "clear sky",
            "icon": "01d",
            "id": 800,
            "main": "Clear"
        }
    ],
    "wind": {
        "deg": 50,
        "speed": 15.66
    }
}
{
    "base": "stations",
    "clouds": {
        "all": 99
    },
    "cod": 200,
    "coord": {
        "lat": -1.6792,
        "lon": 29.2228
    },
    "dt": 1644545699,
    "id": 216281

{
    "cod": "404",
    "message": "city not found"
}
{
    "base": "stations",
    "clouds": {
        "all": 1
    },
    "cod": 200,
    "coord": {
        "lat": 28.4667,
        "lon": -107.3
    },
    "dt": 1644545886,
    "id": 3976161,
    "main": {
        "feels_like": 42.91,
        "grnd_level": 792,
        "humidity": 31,
        "pressure": 1017,
        "sea_level": 1017,
        "temp": 46.92,
        "temp_max": 46.92,
        "temp_min": 46.92
    },
    "name": "Adolfo L\u00f3pez Mateos",
    "sys": {
        "country": "MX",
        "id": 2032478,
        "sunrise": 1644501076,
        "sunset": 1644540944,
        "type": 2
    },
    "timezone": -25200,
    "visibility": 10000,
    "weather": [
        {
            "description": "clear sky",
            "icon": "01n",
            "id": 800,
            "main": "Clear"
        }
    ],
    "wind": {
        "deg": 104,
        "gust": 8.75,
        "speed": 8.25
    }
}
{
    "base": "stations",
    "clouds": {

{
    "base": "stations",
    "clouds": {
        "all": 100
    },
    "cod": 200,
    "coord": {
        "lat": -21.6403,
        "lon": -41.0511
    },
    "dt": 1644545432,
    "id": 3448903,
    "main": {
        "feels_like": 73.17,
        "grnd_level": 1012,
        "humidity": 97,
        "pressure": 1013,
        "sea_level": 1013,
        "temp": 71.73,
        "temp_max": 71.73,
        "temp_min": 71.73
    },
    "name": "S\u00e3o Jo\u00e3o da Barra",
    "sys": {
        "country": "BR",
        "sunrise": 1644481892,
        "sunset": 1644528329
    },
    "timezone": -10800,
    "visibility": 10000,
    "weather": [
        {
            "description": "overcast clouds",
            "icon": "04n",
            "id": 804,
            "main": "Clouds"
        }
    ],
    "wind": {
        "deg": 217,
        "gust": 11.34,
        "speed": 8.39
    }
}
{
    "base": "stations",
    "clouds": {
        "all": 100
    },
    "cod": 200,
    "coord": {
        "lat": 39.816

{
    "base": "stations",
    "clouds": {
        "all": 67
    },
    "cod": 200,
    "coord": {
        "lat": 29.9737,
        "lon": 32.5263
    },
    "dt": 1644545893,
    "id": 359796,
    "main": {
        "feels_like": 52.03,
        "grnd_level": 1019,
        "humidity": 59,
        "pressure": 1020,
        "sea_level": 1020,
        "temp": 54.14,
        "temp_max": 54.14,
        "temp_min": 54.14
    },
    "name": "Suez",
    "sys": {
        "country": "EG",
        "sunrise": 1644554008,
        "sunset": 1644593698
    },
    "timezone": 7200,
    "visibility": 10000,
    "weather": [
        {
            "description": "broken clouds",
            "icon": "04n",
            "id": 803,
            "main": "Clouds"
        }
    ],
    "wind": {
        "deg": 302,
        "gust": 8.41,
        "speed": 5.93
    }
}
{
    "base": "stations",
    "clouds": {
        "all": 100
    },
    "cod": 200,
    "coord": {
        "lat": 43.2001,
        "lon": -75.4663
    }

{
    "base": "stations",
    "clouds": {
        "all": 0
    },
    "cod": 200,
    "coord": {
        "lat": -29.038,
        "lon": 24.6028
    },
    "dt": 1644545897,
    "id": 961226,
    "main": {
        "feels_like": 59.52,
        "grnd_level": 891,
        "humidity": 39,
        "pressure": 1011,
        "sea_level": 1011,
        "temp": 61.81,
        "temp_max": 61.81,
        "temp_min": 61.81
    },
    "name": "Ritchie",
    "sys": {
        "country": "ZA",
        "id": 1928,
        "sunrise": 1644551999,
        "sunset": 1644599510,
        "type": 1
    },
    "timezone": 7200,
    "visibility": 10000,
    "weather": [
        {
            "description": "clear sky",
            "icon": "01n",
            "id": 800,
            "main": "Clear"
        }
    ],
    "wind": {
        "deg": 214,
        "gust": 3.11,
        "speed": 2.53
    }
}
{
    "base": "stations",
    "clouds": {
        "all": 97
    },
    "cod": 200,
    "coord": {
        "lat": 53.7

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [4]:
#  Get the indices of cities that have humidity over 100%.


In [5]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression